In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf

In [10]:
def load_satellite(num_training=4035, num_validation=400, num_test=2000):
    train = np.loadtxt('data/sat_train.txt', dtype='float32')
    test = np.loadtxt('data/sat_test.txt', dtype='float32')
    
    np.random.seed(10)
    np.random.shuffle(train)
    np.random.shuffle(test)
    
    X_train, y_train = train[:,:36],train[:,36]
    X_test, y_test = test[:, :36], test[:,36]
    
    y_train = y_train.astype('int')
    y_test = y_test.astype('int')
    
    train_label7 = y_train == 7
    y_train[train_label7] = 6
    y_train -= 1
    
    test_label7 = y_test == 7
    y_test[test_label7] = 6
    y_test -= 1
    
    # Subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    return X_train, y_train, X_val, y_val, X_test, y_test
X_train, y_train, X_val, y_val, X_test, y_test = load_satellite()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (4035, 36)
Train labels shape:  (4035,) int64
Validation data shape:  (400, 36)
Validation labels shape:  (400,)
Test data shape:  (2000, 36)
Test labels shape:  (2000,)


In [19]:
# This is to expedite the process 
train_subset = 4035
# This is a good beta value to start with
beta = 0.01
num_features = 36
num_hidden = 10
num_labels = 6


graph = tf.Graph()
with graph.as_default():

    # Input data.
    # They're all constants.
    tf_train_dataset = tf.constant(X_train[:train_subset, :])
    tf_train_labels = tf.constant(y_train[:train_subset])
    tf_valid_dataset = tf.constant(X_val)
    tf_test_dataset = tf.constant(X_test)
  
    # Variables    
    # They are variables we want to update and optimize.
    w1 = tf.Variable(tf.truncated_normal([num_features, num_hidden]))
    b1 = tf.Variable(tf.zeros([num_hidden]))
    h1 = tf.nn.sigmoid(tf.matmul(tf_train_dataset, w1) + b1)
    
    w2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    # Training computation.
    logits = tf.matmul(h1, w2) + b2 
    # Original loss function
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels) )
    # Loss function using L2 Regularization
    regularizer = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
#     valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, w1) + biases )
#     test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step {}: {}'.format(step, l))
            print('Training accuracy: {:.1f}'.format(accuracy(predictions, 
                                                         train_labels[:train_subset, :])))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            
            # You don't have to do .eval above because we already ran the session for the
            # train_prediction
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), 
                                                           valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels))) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: 3.6005706787109375


NameError: name 'train_labels' is not defined